In [1]:
!pip install python-terrier
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_colbert.git
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.0/318.0 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import faiss
import pyterrier as pt
pt.init()

terrier-assemblies 5.7 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.7 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



In [4]:
checkpoint="http://www.dcs.gla.ac.uk/~craigm/colbert.dnn.zip"

In [19]:
import numpy as np
import pandas as pd
import nltk
import json
import re
import csv
from tqdm import tqdm
def load_data():
  data = []
  with open("/content/gdrive/MyDrive/MLProject/booksummaries.txt", 'r') as f:
    reader = csv.reader(f, dialect='excel-tab')
    for row in tqdm(reader):
      data.append(row)

  book_index = []
  book_id = []
  book_author = []
  book_name = []
  summary = []
  genre = []
  a = 1
  for i in tqdm(data):
      book_index.append(a)
      a = a+1
      book_id.append(i[0])
      book_name.append(i[2])
      book_author.append(i[3])
      genre.append(i[5])
      summary.append(i[6])
  df = pd.DataFrame({'docno': book_index, 'ID': book_id, 'BookTitle': book_name, 'Author': book_author,
                        'Genre': genre, 'Summary': summary})
  def clean_data(df):
    df.isna().sum()
    df = df.drop(df[df['Genre'] == ''].index)
    df = df.drop(df[df['Summary'] == ''].index)
    genres_cleaned = []
    for i in df['Genre']:
        genres_cleaned.append(list(json.loads(i).values()))
    df['Genre'] = genres_cleaned

    def clean_summary(text):
      text = re.sub("\'", "", text)
      text = re.sub("[^a-zA-Z]"," ",text)
      text = ' '.join(text.split())
      text = text.lower()
      return text
    df['text'] = df['Summary'].apply(lambda x: clean_summary(x))
    empty_index = df[df['text'] == ''].index
    df = df.drop(empty_index)
    return df
  df = clean_data(df)
  return df

# load document set
df = load_data()

16559it [00:00, 18429.19it/s]
100%|██████████| 16559/16559 [00:00<00:00, 597660.08it/s]


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12836 entries, 0 to 16558
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   docno      12836 non-null  int64 
 1   ID         12836 non-null  object
 2   BookTitle  12836 non-null  object
 3   Author     12836 non-null  object
 4   Genre      12836 non-null  object
 5   Summary    12836 non-null  object
 6   text       12836 non-null  object
dtypes: int64(1), object(6)
memory usage: 802.2+ KB


In [21]:
df[df['text']==''].index

Int64Index([], dtype='int64')

In [22]:
preprocessed_data = df.to_dict('records')

In [23]:
preprocessed_data[3382]

{'docno': 4047,
 'ID': '2833878',
 'BookTitle': 'Junk',
 'Author': 'Melvin Burgess',
 'Genre': ['Literary realism',
  "Children's literature",
  'Young adult literature'],
 'Summary': " The novel is told in the first person from various points of view; much of the narration is by Gemma and Tar, and takes place over the course of a few years in their lives. At the beginning of the story, Gemma is spending time with Tar, a gentle teenager who is planning to run away from home after suffering abuse at the hands of his father. Gemma is a rebellious 14 year old and, despite having loving parents, decides to leave home and join Tar in Bristol. In Bristol, Tar is introduced to Richard through Skolly, the tobacconist that took pity on Tar. Richard is a nervous, vegetarian anarchist, and he and his friends Vonny, a motherly character, and Vonny's boyfriend Jerry break into an abandoned house to squat there and invite Tar to stay with them. Tar is eager for them to meet Gemma as she arrives in B

In [24]:
!rm -rf /content/gdrive/MyDrive/MLProject/colbertindex

import pyterrier_colbert.indexing

indexer = pyterrier_colbert.indexing.ColBERTIndexer(checkpoint, "/content/gdrive/MyDrive/MLProject/", "colbertindex", chunksize=3)
indexer.index(preprocessed_data)

[Jun 18, 13:58:28] [0] 		 #> Local args.bsize = 128
[Jun 18, 13:58:28] [0] 		 #> args.index_root = /content/gdrive/MyDrive/MLProject/
[Jun 18, 13:58:28] [0] 		 #> self.possible_subset_sizes = [69905]


Some weights of the model checkpoint at bert-base-uncased were not used when initializing ColBERT: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ColBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ColBERT were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['linear.weight']
You should probably TRAIN this model on a down-stream task

[Jun 18, 13:58:30] #> Loading model checkpoint.
[Jun 18, 13:58:30] #> Loading checkpoint http://www.dcs.gla.ac.uk/~craigm/colbert.dnn.zip


/usr/local/lib/python3.10/dist-packages/torch/hub.py:665: UserWarning: Falling back to the old format < 1.6. This support will be deprecated in favor of default zipfile format introduced in 1.6. Please redo torch.save() to save it in the new zipfile format.
  warnings.warn('Falling back to the old format < 1.6. This support will be '


[Jun 18, 13:58:45] #> checkpoint['epoch'] = 0
[Jun 18, 13:58:45] #> checkpoint['batch'] = 44500




[Jun 18, 13:58:45] #> Note: Output directory /content/gdrive/MyDrive/MLProject/ already exists




[Jun 18, 13:58:45] #> Creating directory /content/gdrive/MyDrive/MLProject/colbertindex 


[Jun 18, 14:01:10] [0] 		 #> Completed batch #0 (starting at passage #0) 		Passages/min: 5.3k (overall),  5.3k (this encoding),  19577.5M (this saving)
[Jun 18, 14:01:11] [0] 		 [NOTE] Done with local share.
[Jun 18, 14:01:11] [0] 		 #> Joining saver thread.
[Jun 18, 14:01:11] [0] 		 #> Saved batch #0 to /content/gdrive/MyDrive/MLProject/colbertindex/0.pt 		 Saving Throughput = 453.3k passages per minute.

#> num_embeddings = 1984112
[Jun 18, 14:01:11] #> Starting..
[Jun 18, 14:01:11] #> Processing slice #1 of 1 (range 0..1).
[Jun 18, 14:01:11] #> Will write to /content/gdrive/MyDrive/MLProject/colbertindex/ivfpq.100.faiss.
[Jun 18, 14:01:11] #> Loading /content/gdrive/MyDrive/MLProject/colbertindex/0

In [26]:
pyterrier_colbert_factory = pyterrier_colbert.ranking.ColBERTFactory(checkpoint, "/content/gdrive/MyDrive/MLProject/", "colbertindex")
colbert_e2e = pyterrier_colbert_factory.end_to_end()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing ColBERT: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ColBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ColBERT were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['linear.weight']
You should probably TRAIN this model on a down-stream task

[Jun 18, 14:05:01] #> Loading model checkpoint.
[Jun 18, 14:05:01] #> Loading checkpoint http://www.dcs.gla.ac.uk/~craigm/colbert.dnn.zip
[Jun 18, 14:05:42] #> checkpoint['epoch'] = 0
[Jun 18, 14:05:42] #> checkpoint['batch'] = 44500
[Jun 18, 14:05:42] #> Loading the FAISS index from /content/gdrive/MyDrive/MLProject/colbertindex/ivfpq.faiss ..
[Jun 18, 14:05:42] #> Building the emb2pid mapping..
[Jun 18, 14:05:42] len(self.emb2pid) = 1984112
Loading reranking index, memtype=mem


Loading index shards to memory:   0%|          | 0/1 [00:00<?, ?shard/s]

In [28]:
results = (colbert_e2e % 10).search("harry potter")


In [32]:
df.loc[df['docno'].isin(results['docno'].astype('int64').tolist())]

,docno,ID,BookTitle,Author,Genre,Summary,text
445,446,156489,Harry Potter and the Order of the Phoenix,J. K. Rowling,"[Fantasy, Young adult literature, Fiction]",Harry Potter is spending another summer with ...,harry potter is spending another summer with h...
480,481,163234,Harry Potter and the Chamber of Secrets,J. K. Rowling,"[Speculative fiction, Fantasy, Fiction]",Harry Potter and the Chamber of Secrets begin...,harry potter and the chamber of secrets begins...
687,688,241855,Fantastic Beasts and Where to Find Them,J. K. Rowling,[Speculative fiction],Fantastic Beasts purports to be a reproductio...,fantastic beasts purports to be a reproduction...
724,725,259128,Harry Potter and the Goblet of Fire,J. K. Rowling,[Speculative fiction],The story begins in the 1940s in a small town...,the story begins in the s in a small town call...
748,749,272678,Quidditch Through the Ages,J. K. Rowling,[Speculative fiction],In 2001 Rowling penned two companion books to...,in rowling penned two companion books to the h...
4326,4327,3116503,Rabbit Redux,John Updike,"[Fiction, Novel]",Rabbit Redux finds the former high-school bas...,rabbit redux finds the former high school bask...
4471,4472,3280309,Signal to Noise,Eric S. Nylund,"[Cyberpunk, Science Fiction, Speculative fiction]","The novel follows Jack Potter, a computer cry...",the novel follows jack potter a computer crypt...
5622,5623,4535237,The Tale of the Pie and the Patty-Pan,Beatrix Potter,"[Children's literature, Fiction]",Potter declared the tale her next favourite t...,potter declared the tale her next favourite to...
10958,10959,13847930,Nevermore,,"[Mystery, Fiction]",Harry Houdini joins forces with Arthur Conan ...,harry houdini joins forces with arthur conan d...
15182,15183,27142770,Necroscope,Brian Lumley,"[Science Fiction, Adventure novel]","Harry is an English youth in school, and stra...",harry is an english youth in school and strang...
